In [21]:
from predictNew import PreprocessData, Predict
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
data = PreprocessData()

In [17]:
total_df = data.transform('data/dsjobs_training_culled.csv', 'data/ethan_profile.csv')

In [22]:
vec = TfidfVectorizer(stop_words='english')

In [24]:
vec.fit([total_df.iloc[1,1]])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [25]:
vec.vocabulary_

{'interested': 137,
 'working': 298,
 'exciting': 91,
 'products': 204,
 'microsoft': 165,
 'passionate': 187,
 'exceeding': 89,
 'customer': 64,
 'expectations': 92,
 'advancing': 5,
 'cloud': 41,
 'strategy': 274,
 'start': 269,
 'like': 150,
 'environment': 86,
 'excited': 90,
 'computing': 50,
 'technology': 279,
 'driving': 76,
 'growth': 115,
 'core': 58,
 'businesses': 31,
 'look': 154,
 'azure': 21,
 'experience': 93,
 'cxp': 66,
 'team': 277,
 'provides': 210,
 'customers': 65,
 'demand': 69,
 'infinitely': 131,
 'scalable': 242,
 'infrastructure': 133,
 'platform': 193,
 'build': 28,
 'host': 120,
 'scale': 243,
 'service': 254,
 'applications': 16,
 'internet': 138,
 'global': 112,
 'data': 67,
 'centers': 35,
 'engineering': 84,
 'organization': 182,
 'rapidly': 217,
 'growing': 114,
 'committed': 45,
 'relentless': 228,
 'pursuit': 211,
 'satisfied': 240,
 'leading': 146,
 'world': 299,
 'class': 38,
 'reliability': 229,
 'engagements': 83,
 'modern': 169,
 'experiences': 